In [1]:
import visual_behavior_glm.src.GLM_params as glm_params
import visual_behavior_glm.src.GLM_analysis_tools as gat
import visual_behavior_glm.src.GLM_visualization_tools as gvt
from visual_behavior_glm.src.glm import GLM
import matplotlib.pyplot as plt
import visual_behavior.data_access.loading as loading
import visual_behavior.database as db
import plotly.express as px

import pandas as pd
import numpy as np
import os

import seaborn as sns

import visual_behavior.plotting as vbp

import plotly.express as px
from sklearn.cluster import KMeans
import umap

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
%matplotlib notebook
%widescreen

applied a custom magic command to make full use of screen width
will only work if command is defined locally
replace with the following to replicate functionality: 
	from IPython.core.display import display, HTML
	display(HTML("<style>.container { width:100% !important; }</style>")


# Gather/organize data

## load the results summary for a single GLM version from mongo

In [3]:
glm_version = '4_L2_optimize_by_cell'
dropout_summary = gat.retrieve_results(search_dict = {'glm_version': glm_version}, results_type='summary')
dropout_summary.sample(5)

,_id,index,dropout,variance_explained,fraction_change_from_full,absolute_change_from_full,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,entry_time_utc
937956,5f22480e0997a3cb6adb7aa6,4,post_licks,0.631803,-0.000006,-0.000004,962579279,794381992,793857113,794071128,814796612,VisualBehavior,container_qc,passed,20181212_412366_3imagesA,OPHYS_3_images_A,CAM2P.4,2018-12-12 16:00:43.000000,765187424,756674785,F,149.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,756674785,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,2020-07-31 18:21:02.812807
701808,5f2238ad03983b7544a5ade9,16,pupil,0.108933,0.001861,0.000202,1007037568,968651957,968445235,968564134,957361481,VisualBehaviorTask1B,container_qc,passed,20191016_479458_6imagesA,OPHYS_6_images_A,CAM2P.4,2019-10-16 20:25:03.000000,928775322,916634005,M,126.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,916634005,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,2020-07-31 18:17:35.650554
1157484,5f245fcc1bdc0ee0ce5af06c,12,correct_rejects,0.000072,-1.076203,0.001017,982805540,974384470,974019098,974140453,968451510,VisualBehavior,container_qc,passed,20191030_479426_3imagesAretake,OPHYS_3_images_A,CAM2P.5,2019-10-30 15:40:18.000000,926756680,916458892,F,141.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,['Ai94(TITL-GCaMP6s)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,916458892,Slc17a7-IRES2-Cre,NaN,NaN,1,1,Slc17a7_VISp_175,4_L2_optimize_by_cell,2020-07-31 18:15:40.423695
1262131,5f24613ca9cb12cdf91910f6,16,pupil,0.016725,2.623114,0.012109,961884624,823392290,822734832,822995693,803516615,VisualBehavior,container_qc,passed,20190214_423606_3imagesa,OPHYS_3_images_A,CAM2P.3,2019-02-14 16:07:54.000000,774563995,766015390,M,152.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,['Ai94(TITL-GCaMP6s)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,766015390,Slc17a7-IRES2-Cre,NaN,NaN,1,1,Slc17a7_VISp_175,4_L2_optimize_by_cell,2020-07-31 18:21:48.769888
1030730,5f245ecfdd8fd70bacf21204,13,omissions,0.080593,0.017296,0.001370,1007048423,957655294,957247123,957387951,939523510,VisualBehavior,container_qc,passed,20190927_457766_6imagesB,OPHYS_6_images_B,CAM2P.3,2019-09-27 19:13:06.000000,856448219,847864095,M,213.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,['Ai94(TITL-GCaMP6s)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,847864095,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_175,4_L2_optimize_by_cell,2020-07-31 18:11:27.488756


In [4]:
dropout_summary_pivoted = gat.build_pivoted_results_summary(results_summary=dropout_summary, cutoff=0.01, value_to_use='fraction_change_from_full')

## load the behavior model weights

In [5]:
behavior_model_summary = loading.get_behavior_model_summary_table()
behavior_model_summary.sample(10)

,behavior_session_id,ophys_session_id,ophys_experiment_id,project_code,session_type,equipment_name,date_of_acquisition,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,cre_line,donor_id,active,imaging_depth,container_id,image_set,trained_A,trained_B,stage,good,num_hits,num_fa,num_cr,num_miss,num_aborts,session_roc,lick_fraction,lick_fraction_1st,lick_fraction_2nd,lick_hit_fraction,lick_hit_fraction_1st,lick_hit_fraction_2nd,trial_hit_fraction,trial_hit_fraction_1st,trial_hit_fraction_2nd,task_dropout_index,task_only_dropout_index,timing_only_dropout_index,prior_bias,avg_weight_bias,avg_weight_bias_1st,avg_weight_bias_2nd,prior_omissions1,avg_weight_omissions1,avg_weight_omissions1_1st,avg_weight_omissions1_2nd,prior_task0,avg_weight_task0,avg_weight_task0_1st,avg_weight_task0_2nd,prior_timing1D,avg_weight_timing1D,avg_weight_timing1D_1st,avg_weight_timing1D_2nd,task_weight_index,task_weight_index_1st,task_weight_index_2nd,task_session,container_in_order,full_container,meso
180,842729895,842583486,[842973730],VisualBehavior,OPHYS_1_images_A,CAM2P.3,2019-03-28 15:02:48.000000,813703544,M,108.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",Slc17a7-IRES2-Cre,813703535,True,375.0,843049827.0,A,True,False,1,True,82.0,8.0,10.0,44.0,638.0,0.869731,0.178262,0.180952,0.175636,0.113889,0.113573,0.114206,0.650794,0.640625,0.661290,-33.089781,-0.250295,-33.340076,0.166195,-2.475973,-2.507678,-2.445028,0.001492,0.944840,0.946835,0.942893,0.000849,0.875134,0.874478,0.875775,0.085161,5.372809,5.791027,4.964617,-4.497675,-4.916549,-4.088842,False,True,False,NaN
76,908122381,907991198,[908381652 908381654 908381656 908381658 90838...,VisualBehaviorMultiscope,OPHYS_6_images_B,MESO.1,2019-07-17 11:26:37.671800,843122504,F,146.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",Slc17a7-IRES2-Cre,843122488,True,NaN,NaN,NaN,NaN,NaN,6,True,146.0,2.0,28.0,61.0,546.0,0.836106,0.168441,0.211434,0.126961,0.209770,0.158508,0.292135,0.705314,0.781609,0.650000,-1.091783,-9.060804,-10.152587,0.069118,-2.277121,-1.595093,-2.935150,0.016933,2.153090,2.452861,1.863868,0.111702,2.638096,2.702120,2.576325,0.065509,3.151334,3.188714,3.115269,-0.513238,-0.486594,-0.538944,False,NaN,NaN,True
193,964894591,964691111,[965218395],VisualBehaviorTask1B,OPHYS_6_images_A,CAM2P.5,2019-10-10 15:52:31.000000,912554454,F,135.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",Slc17a7-IRES2-Cre,912554442,True,175.0,947838486.0,A,False,True,6,True,157.0,4.0,33.0,81.0,471.0,0.759861,0.162095,0.189933,0.136792,0.239329,0.234973,0.244828,0.659664,0.774775,0.559055,11.879859,-13.269371,-1.389512,0.036270,-1.934600,-1.713242,-2.135806,0.005411,1.416543,1.356364,1.471243,0.028231,3.238928,3.667730,2.849163,0.082676,-0.767023,-1.021342,-0.535856,4.005951,4.689072,3.385019,True,True,True,NaN
42,849098851,848983781,[849233390 849233392 849233394 849233396 84923...,VisualBehaviorMultiscope,OPHYS_4_images_B,MESO.1,2019-04-10 11:22:28.314537,810573072,M,129.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],Vip-IRES-Cre,810573033,True,NaN,NaN,NaN,NaN,NaN,4,True,97.0,3.0,38.0,183.0,219.0,0.906026,0.072781,0.142996,0.012175,0.300310,0.295918,0.344828,0.346429,0.820755,0.057471,3.519637,-12.210231,-8.690594,0.068895,-3.726504,-1.690770,-5.483627,0.009113,1.815500,1.880089,1.759750,0.021665,2.763763,3.236857,2.355417,0.140423,-0.314531,2.705443,-2.921193,3.078294,0.531414,5.276610,True,NaN,NaN,True
21,878120101,877946125,[878363088],VisualBehavior,OPHYS_3_images_A,CAM2P.3,2019-05-30 17:50:27.000000,834823477,F,125.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",Slc17a7-IRES2-Cre,834823464,True,175.0,869781307.0,A,True,False,3,True,138.0,7.0,18.0,40.0,518.0,0.805884,0.154659,0.167600,0

In [6]:
fig,ax=plt.subplots()
ax.hist(behavior_model_summary['task_dropout_index'])
ax.set_xlabel('task_dropout_index')

<IPython.core.display.Javascript object>

Text(0.5, 0, 'task_dropout_index')

## get the dominant dropout for each cell in each session

In [7]:
def get_dominant_dropout(row, cols_to_check):
    dominant_dropout = None
    dominant_dropout_value = 0
    for col in cols_to_check:
        if row[col] < dominant_dropout_value:
            dominant_dropout = col
            dominant_dropout_value = row[col]
    return dominant_dropout

cols_to_check = ['all-images','omissions','running','pupil']
dropout_summary_pivoted['dominant_dropout'] = dropout_summary_pivoted.apply(get_dominant_dropout, axis=1, args=[cols_to_check])

In [8]:
dropout_summary_pivoted['dominant_dropout'].unique()

array(['all-images', 'omissions', 'running', 'pupil', None], dtype=object)

## merge in task dropout index

In [9]:
dropout_summary_pivoted = dropout_summary_pivoted.merge(
    behavior_model_summary[['ophys_session_id','task_dropout_index']],
    left_on = 'ophys_session_id',
    right_on = 'ophys_session_id',
    how='left'
)

In [10]:
dropout_summary_pivoted.sample(10)

,identifier,Full,all-images,beh_model,change,correct_rejects,false_alarms,hits,image0,image1,image2,image3,image4,image5,image6,image7,image_expectation,intercept,misses,model_bias,model_omissions1,model_task0,model_timing1D,omissions,post_lick_bouts,post_licks,pre_lick_bouts,pre_licks,pupil,rewards,running,time,visual,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,dominant_dropout,task_dropout_index
19308,956941846_1028820630,0.0,-0.048428,-0.002014,0.000103,0.001070,0.001277,0.000282,-0.005380,-0.002196,-0.002595,-0.001664,-0.001385,-0.000292,-0.004679,-0.002415,0.000296,-0.000488,-0.000527,0.001102,-0.000143,-0.002795,0.001242,-0.067400,-0.006079,-0.001013,0.012721,0.005542,-0.119280,-0.005769,-0.140596,0.000963,-0.302513,1028820630,956941846,955775716,956010809,1018028339,VisualBehaviorMultiscope,completed,passed,20190926_457841_3imagesA,OPHYS_3_images_A,MESO.1,2019-09-26 08:44:37.199778,858992726,850862430,F,212.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],75,VISp,NaN,850862430,Sst-IRES-Cre,NaN,"parent_averaged_depth_image_6,parent_averaged_...",1,1,Sst_VISp_75,4_L2_optimize_by_cell,running,-10.801216
19060,956820482_1007049430,0.0,-2.167741,0.511890,0.001324,0.007626,0.013609,0.023174,0.008549,-0.009112,-0.001769,-0.009894,0.001933,-0.005090,-0.010882,0.001758,0.003775,0.023783,0.003560,0.312575,0.091453,0.137665,0.082962,0.007514,0.003681,0.041710,-0.005614,0.023510,-0.056594,0.036314,-0.024344,-0.088368,-2.177916,1007049430,956820482,956121729,956269097,939523510,VisualBehavior,container_qc,passed,20190926_457766_4imagesB,OPHYS_4_images_B,CAM2P.3,2019-09-26 18:32:22.000000,856448219,847864095,M,212.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,['Ai94(TITL-GCaMP6s)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,847864095,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_175,4_L2_optimize_by_cell,all-images,-9.052628
14343,905955204_1028792533,0.0,-0.072001,0.066743,0.002531,0.090499,0.041024,0.027195,0.038583,0.038654,-0.011603,0.014747,-0.013237,0.038847,0.004863,0.030005,0.037535,0.016642,0.036245,0.009916,0.014623,0.003292,0.043344,0.127714,-0.028952,0.078729,0.014490,0.286210,-0.217778,0.151827,-0.220965,0.033516,-0.006227,1028792533,905955204,904771513,904922800,1018027650,VisualBehaviorMultiscope,completed,passed,20190712_456915_Ophys3,OPHYS_3_images_A,MESO.1,2019-07-12 12:22:04.270010,852260338,843122504,F,141.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",275,VISp,NaN,843122504,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_275,4_L2_optimize_by_cell,running,-7.577298
7177,848697604_962576808,0.0,-0.075029,-0.024804,0.000190,0.008856,0.000296,0.003715,-0.002645,0.001142,-0.001187,0.001055,0.000565,-0.000930,-0.007230,0.000967,-0.002573,-0.001769,0.002586,0.004422,0.008263,0.010302,0.024411,0.009224,0.000703,-0.013443,0.008429,0.010360,-0.062126,0.007836,-0.201071,0.004617,-0.097637,962576808,848697604,848143026,848348215,845588020,VisualBehavior,container_qc,passed,20190409_445270_1imagesA,OPHYS_1_images_A,CAM2P.3,2019-04-09 16:16:45.000000,828775962,820871408,M,96.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,820871408,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,running,2.337763
13579,903405627_962553077,0.0,-0.886721,0.003577,0.000182,0.003639,0.005569,0.002225,-0.110090,0.001508,-0.008424,0.001445,0.000807,0.000952,0.001205,0.001346,0.000454,0.000262,0.001241,0.000609,-0.000047,0.000192,

## add a `session_id` column with a numeric value for the session_type (to lump together sessions by order, regardless of image set)

In [11]:
def map_session_types(session_type):
    session_id = int(session_type[6:7])
    return session_id
dropout_summary_pivoted['session_id'] = dropout_summary_pivoted['session_type'].map(lambda st:map_session_types(st))

dropout_summary_pivoted

,identifier,Full,all-images,beh_model,change,correct_rejects,false_alarms,hits,image0,image1,image2,image3,image4,image5,image6,image7,image_expectation,intercept,misses,model_bias,model_omissions1,model_task0,model_timing1D,omissions,post_lick_bouts,post_licks,pre_lick_bouts,pre_licks,pupil,rewards,running,time,visual,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,dominant_dropout,task_dropout_index,session_id
0,1000743008_1003146172,0.0,-0.140223,-0.001848,0.000581,0.014875,0.008197,0.005280,-0.000634,0.003108,0.002692,0.001819,0.003621,-0.001186,-0.026248,0.002645,0.000947,0.000913,-0.001280,-0.001924,0.000501,-0.000538,0.000135,0.008544,0.007587,-0.005937,0.011424,0.014952,-0.074100,0.010710,-0.020992,-0.001004,-0.228425,1003146172,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
1,1000743008_1003146180,0.0,-0.155019,0.004636,0.000530,0.020636,0.011811,0.003584,0.002961,0.003979,-0.007203,-0.007210,0.002283,-0.018611,0.002152,0.002657,0.000976,0.000081,0.006030,0.000728,0.001277,0.000856,0.002049,0.020436,0.001011,0.003103,0.031004,0.037226,0.005513,0.032333,-0.002625,0.000434,-0.252412,1003146180,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
2,1000743008_1003146183,0.0,-0.210215,0.062499,0.000224,0.002435,0.001871,0.001834,0.001881,0.002758,-0.015761,0.000383,0.000302,-0.012279,0.008475,0.001058,-0.001232,-0.002156,0.000572,0.010619,0.029392,-0.001927,0.021750,-0.000316,-0.004966,0.002149,0.003036,0.008150,-0.057209,-0.000740,0.014037,0.004291,-0.319959,1003146183,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
3,1000743008_1003146186,0.0,-0.368535,0.021722,0.000232,0.002308,0.006869,0.002990,-0.001212,-0.003161,0.000206,-0.049498,0.001615,-0.021775,0.000046,0.002221,-0.003134,0.003374,0.002284,0.014580,0.006673,0.005393,-0.001058,0.002760,-0.001018,0.000574,0.004443,0.004108,0.005936,0.011120,0.001867,0.002495,-0.435852,1003146186,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
4,1000743008_1003146190,0.0,-0.089674,0.000823,0.000713,0.027508,0.019429,0.006967,0.006088,0.004045,-0.003198,-0.024926,0.003739,0.002454,-0.000550,0.008610,0.001467,0.000038,0.000181,-0.000054,0.001188,0.000657,-0.001699,0.031173,0.010330,0.005812,0.031873,0.030949,-0.019613,0.021468,0.005709,0.000490,-0.184349,1003146

In [12]:
dropout_summary_pivoted.columns

Index(['identifier', 'Full', 'all-images', 'beh_model', 'change',
       'correct_rejects', 'false_alarms', 'hits', 'image0', 'image1', 'image2',
       'image3', 'image4', 'image5', 'image6', 'image7', 'image_expectation',
       'intercept', 'misses', 'model_bias', 'model_omissions1', 'model_task0',
       'model_timing1D', 'omissions', 'post_lick_bouts', 'post_licks',
       'pre_lick_bouts', 'pre_licks', 'pupil', 'rewards', 'running', 'time',
       'visual', 'cell_specimen_id', 'ophys_experiment_id', 'ophys_session_id',
       'behavior_session_id', 'container_id', 'project_code',
       'container_workflow_state', 'experiment_workflow_state', 'session_name',
       'session_type', 'equipment_name', 'date_of_acquisition',
       'isi_experiment_id', 'specimen_id', 'sex', 'age_in_days',
       'full_genotype', 'reporter_line', 'driver_line', 'imaging_depth',
       'targeted_structure', 'published_at', 'super_container_id', 'cre_line',
       'session_tags', 'failure_tags', 'exposu

In [13]:
dropout_summary_pivoted.equipment_name.unique()

array(['CAM2P.3', 'CAM2P.4', 'CAM2P.5', 'MESO.1'], dtype=object)

## Turn some categorical columns into numerical columns

In [14]:
def make_categorical(df, column):
    df['{}_categorical'.format(column)] = pd.Categorical(df[column], ordered=True).codes

for column in ['cre_line','equipment_name','targeted_structure','session_id','dominant_dropout']:
    make_categorical(dropout_summary_pivoted, column)
    
dropout_summary_pivoted

,identifier,Full,all-images,beh_model,change,correct_rejects,false_alarms,hits,image0,image1,image2,image3,image4,image5,image6,image7,image_expectation,intercept,misses,model_bias,model_omissions1,model_task0,model_timing1D,omissions,post_lick_bouts,post_licks,pre_lick_bouts,pre_licks,pupil,rewards,running,time,visual,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,dominant_dropout,task_dropout_index,session_id,cre_line_categorical,equipment_name_categorical,targeted_structure_categorical,session_id_categorical,dominant_dropout_categorical
0,1000743008_1003146172,0.0,-0.140223,-0.001848,0.000581,0.014875,0.008197,0.005280,-0.000634,0.003108,0.002692,0.001819,0.003621,-0.001186,-0.026248,0.002645,0.000947,0.000913,-0.001280,-0.001924,0.000501,-0.000538,0.000135,0.008544,0.007587,-0.005937,0.011424,0.014952,-0.074100,0.010710,-0.020992,-0.001004,-0.228425,1003146172,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
1,1000743008_1003146180,0.0,-0.155019,0.004636,0.000530,0.020636,0.011811,0.003584,0.002961,0.003979,-0.007203,-0.007210,0.002283,-0.018611,0.002152,0.002657,0.000976,0.000081,0.006030,0.000728,0.001277,0.000856,0.002049,0.020436,0.001011,0.003103,0.031004,0.037226,0.005513,0.032333,-0.002625,0.000434,-0.252412,1003146180,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
2,1000743008_1003146183,0.0,-0.210215,0.062499,0.000224,0.002435,0.001871,0.001834,0.001881,0.002758,-0.015761,0.000383,0.000302,-0.012279,0.008475,0.001058,-0.001232,-0.002156,0.000572,0.010619,0.029392,-0.001927,0.021750,-0.000316,-0.004966,0.002149,0.003036,0.008150,-0.057209,-0.000740,0.014037,0.004291,-0.319959,1003146183,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
3,1000743008_1003146186,0.0,-0.368535,0.021722,0.000232,0.002308,0.006869,0.002990,-0.001212,-0.003161,0.000206,-0.049498,0.001615,-0.021775,0.000046,0.002221,-0.003134,0.003374,0.002284,0.014580,0.006673,0.005393,-0.001058,0.002760,-0.001018,0.000574,0.004443,0.004108,0.005936,0.011120,0.001867,0.002495,-0.435852,1003146186,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
4,1000743008_1003146190,0.0,-0.089674,0.000823,0.000713,0.027508,0.019429,0.006967,0.006088,0.004045,-0.003198,-0.024926,0.003739,0.002454,-0.000550,0.

## category == -1 represents None, set it to NaN

In [15]:
np.sort(dropout_summary_pivoted['dominant_dropout_categorical'].unique())

array([-1,  0,  1,  2,  3], dtype=int8)

In [16]:
dropout_summary_pivoted.loc[dropout_summary_pivoted.query('dominant_dropout_categorical == -1').index,'dominant_dropout_categorical'] = np.nan

## merge in some columns from the original response_summary

In [17]:
dominant_dropout_matrix = (
    dropout_summary_pivoted
    .drop_duplicates(['cell_specimen_id','session_id'])
    .pivot(index='cell_specimen_id',columns='session_id',values='dominant_dropout_categorical')
    .rename(columns={k:'session_{}'.format(k) for k in dropout_summary_pivoted['session_id'].unique()})
)

cols_to_merge = [
    'cell_specimen_id',
    'cre_line',
    'cre_line_categorical',
    'imaging_depth',
    'targeted_structure',
    'targeted_structure_categorical',
    'equipment_name',
    'equipment_name_categorical',
    'task_dropout_index',
]
dominant_dropout_matrix = dominant_dropout_matrix.merge(
    dropout_summary_pivoted[cols_to_merge],
    left_on='cell_specimen_id',
    right_on='cell_specimen_id',
    how='left',
).set_index('cell_specimen_id')
dominant_dropout_matrix

,session_1,session_3,session_4,session_6,cre_line,cre_line_categorical,imaging_depth,targeted_structure,targeted_structure_categorical,equipment_name,equipment_name_categorical,task_dropout_index
cell_specimen_id,,,,,,,,,,,,
817101743,NaN,NaN,NaN,1.0,Vip-IRES-Cre,2,175,VISp,1,CAM2P.4,1,-7.488484
817101771,0.0,0.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,CAM2P.4,1,-4.313725
817101771,0.0,0.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,CAM2P.4,1,-12.645683
817101771,0.0,0.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,CAM2P.4,1,-7.488484
817101837,0.0,1.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,CAM2P.4,1,-4.313725
...,...,...,...,...,...,...,...,...,...,...,...,...
1028894295,0.0,NaN,NaN,NaN,Vip-IRES-Cre,2,81,VISp,1,MESO.1,3,-20.659579
1028894317,2.0,NaN,NaN,NaN,Vip-IRES-Cre,2,81,VISp,1,MESO.1,3,-20.659579
1028894343,0.0,NaN,NaN,NaN,Vip-IRES-Cre,2,81,VISp,1,MESO.1,3,-20.659579


## define plot/heatmap parameters

In [18]:
# make sure there's a heatmap definition for every column, otherwise the column will plot without a heatmap!
cols_to_plot = [
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'cre_line_categorical',
    'targeted_structure_categorical',
    'imaging_depth',
    'equipment_name_categorical',
    'task_dropout_index'
]
heatmap_defs = [
    {
        'columns':['session_{}'.format(i) for i in [1,3,4,6]],
        'cbar_label':'dominant_dropout',
        'cbar_ticks':np.arange(0.5,4.5),
        'cbar_ticklabels':np.sort([c for c in dropout_summary_pivoted['dominant_dropout'].unique() if pd.notnull(c)]),
        'vmin':0,
        'vmax':4,
        'cmap':sns.color_palette("Dark2", 4),
    },
    {
        'columns':['cre_line_categorical'],
        'cbar_label':'cre_line',
        'cbar_ticks':[0,1,2],
        'cbar_ticklabels':np.sort(np.unique(dropout_summary_pivoted['cre_line'])),
        'vmin':-0.5,
        'vmax':2.5,
        'cmap':sns.color_palette("hls", 3),
    },
    {
        'columns':['equipment_name_categorical'],
        'cbar_label':'equipment name',
        'cbar_ticks':np.arange(len(np.unique(dropout_summary_pivoted['equipment_name']))),
        'cbar_ticklabels':np.sort(np.unique(dropout_summary_pivoted['equipment_name'])),
        'vmin':-0.5,
        'vmax':len(np.unique(dropout_summary_pivoted['equipment_name']))-0.5,
        'cmap':sns.color_palette("hls", len(np.unique(dropout_summary_pivoted['equipment_name']))),
    },
    {
        'columns':['targeted_structure_categorical'],
        'cbar_label':'targeted structure',
        'cbar_ticks':np.arange(len(dropout_summary_pivoted['targeted_structure_categorical'].unique())),
        'cbar_ticklabels':np.sort(np.unique(dropout_summary_pivoted['targeted_structure'])),
        'vmin':-0.5,
        'vmax':len(dropout_summary_pivoted['targeted_structure'].unique())-0.5,
        'cmap':sns.color_palette("hls", len(dropout_summary_pivoted['targeted_structure'].unique())),
    },
    {
        'columns':['imaging_depth'],
        'cbar_label':'imaging_depth',
        'cbar_ticks':[0,100,200,300,400],
        'vmin':0,
        'vmax':400,
        'cmap':'magma',
    },
    {
        'columns':['task_dropout_index'],
        'cbar_label':'task_dropout_index',
        'cbar_ticks':[-30,-20,-10,0,10,20,30],
        'vmin':-40,
        'vmax':40,
        'cmap':'RdBu',
    },
]

# plots

## Everything, sorted by cre-line, dropouts in sessions, depth

In [19]:
sort_by = [
    'cre_line_categorical',
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'imaging_depth'
]
data_to_plot = dominant_dropout_matrix[cols_to_plot].sort_values(by=sort_by)
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

## only cells that are matched across all sessions, same sorting as above

In [20]:
sort_by = [
    'cre_line_categorical',
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'imaging_depth'
]
data_to_plot = dominant_dropout_matrix[cols_to_plot].sort_values(by=sort_by).dropna()
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

In [21]:
sort_by = [
    'cre_line_categorical',
    'task_dropout_index',
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'imaging_depth'
]
data_to_plot = dominant_dropout_matrix[cols_to_plot].sort_values(by=sort_by).dropna()
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

## exclude mesoscope sessions, only include cells matched across sessions

In [22]:
sort_by = [
    'cre_line_categorical',
    'imaging_depth'
]
data_to_plot = (
    dominant_dropout_matrix.query('equipment_name != "MESO.1"')[cols_to_plot]
    .sort_values(by=sort_by)
    .dropna()
)
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

## sort only by cre-line and depth (still excluding mesoscope)

In [23]:
sort_by = [
    'cre_line_categorical',
    'imaging_depth',
]
data_to_plot = (
    dominant_dropout_matrix.query('equipment_name != "MESO.1"')[cols_to_plot]
    .sort_values(by=sort_by)
    .dropna()
)
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

## sankey plots

In [24]:
import plotly.graph_objects as go
import itertools

In [25]:
def sankey_diagram(data_to_plot, title_text=''):
    
    # define colormap and categories
    cmap = ['rgb({}, {}, {})'.format(int(255*v[0]),int(255*v[1]),int(255*v[2])) for v in sns.color_palette("Dark2", 4)]
    categories = list(np.sort([c for c in dropout_summary_pivoted['dominant_dropout'].unique() if pd.notnull(c)]))

    # initialize the link dictionary
    link_dict = dict(
        source = [], # indices correspond to labels, eg A1, A2, A2, B1, ...
        target = [],
        value =  [],
        label =  []

    )

    # define values and session IDs
    possible_vals = np.arange(len(categories))
    session_pairs = [(1,3),(3,4),(4,6)]
    # iterate over session pairs:
    for ii, (session_id_1, session_id_2) in enumerate(session_pairs):
        # iterate over all possible combination of dropout values:
        for first_val, second_val in list(itertools.product(possible_vals, possible_vals)):
            # find all rows that meet the condition of having the two sessions equal to the two values
            query = 'session_{} == {} and session_{} == {}'.format(session_id_1, first_val, session_id_2, second_val)
            # update the link dict with relevant data
            link_dict['source'].append(first_val + 4*ii)
            link_dict['target'].append(second_val+ 4*(ii+1))
            link_dict['value'].append(len(data_to_plot.query(query)))
            
            # calculate the fraction of the total, update the label
            label_total = len(data_to_plot.query(query.split(' and')[0]))
            label_fraction = 0 if label_total == 0 else len(data_to_plot.query(query))/label_total
            label = '{} to {} probability: {:0.3f}'.format(categories[first_val], categories[second_val], label_fraction)
            link_dict['label'].append(label)
    
    # add category labels
    category_labels = []
    for session_id in [1,3,4,6]:
        for category_index, category in enumerate(categories):
            query = 'session_{} == {}'.format(session_id, category_index)
            fraction = len(data_to_plot.query(query))/len(data_to_plot)
            category_labels.append('{}\n({:0.1f}%)'.format(category, 100*fraction))

    # define the sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = 4*category_labels,
          color = 4*cmap
        ),
        link = link_dict)])

    # update the layout
    fig.update_layout(title_text=title_text, font_size=10)
    
    # return the figure object:
    return fig

In [26]:
cre_lines = np.sort(dominant_dropout_matrix['cre_line'].unique())
cre_lines

array(['Slc17a7-IRES2-Cre', 'Sst-IRES-Cre', 'Vip-IRES-Cre'], dtype=object)

In [27]:
for cre_line in cre_lines:
    data_to_plot = dominant_dropout_matrix.sort_values(by=sort_by).dropna().query('cre_line == "{}"'.format(cre_line))
    fig = sankey_diagram(data_to_plot, title_text='Sankey plot, GLM verion = {}, cre-line = {}'.format(glm_version, cre_line))
    fig.write_html("/home/dougo/code/dougollerenshaw.github.io/figures_to_share/sankey_diagram_glm_version_{}_cre_line_{}.html".format(glm_version, cre_line))
    fig.show("png")

ValueError: 
The orca executable is required in order to export figures as static images,
but the executable that was found at '/usr/bin/orca'
does not seem to be a valid plotly orca executable. Please refer to the end of
this message for details on what went wrong.

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orca

Alternatively, see other installation methods in the orca project README at
https://github.com/plotly/orca

After installation is complete, no further configuration should be needed.

If you have installed orca, then for some reason plotly.py was unable to
locate it. In this case, set the `plotly.io.orca.config.executable`
property to the full path of your orca executable. For example:

    >>> plotly.io.orca.config.executable = '/path/to/orca'

After updating this executable property, try the export operation again.
If it is successful then you may want to save this configuration so that it
will be applied 